In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import time
from urllib.parse import quote, unquote
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

import pandas as pd
def switch_frame(frame):
    browser.switch_to.default_content()  # frame 초기화
    browser.switch_to.frame(frame)  # frame 변경

# 각 iframe들 [searchIframe / entryIframe / ]

In [24]:
ls2 = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구',' 금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구',
     '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']
# -----------------------------------25개의 구를 한번에 하면 인터넷 문제로 오류가 자주남-----------------------------------
# -----------------------------------5개 구로 쪼개서 하나씩 돌리며 파일을 만듬-----------------------------------
ls = ['강남구']

cafe_name_list = []
cafe_classification_list = []
cafe_review_count_list = []
cafe_address_list = []
cafe_review_list = []
background_image_url_list = []
cafe_menu_ls_list = []

# 처음 모든구를 한번에 돌릴려고 했으나 인터넷문제로 에러가 자주남. for문 제거
# for i in range(len(ls)):  # 모든 자치구에 있는 카페를 알아보기 위해 반복
# 브라우저 열기
keyword = quote(f'{ls[i]} 카페')
url = 'https://map.naver.com/p/search/' + keyword  
browser = webdriver.Chrome()
browser.get(url)
browser.implicitly_wait(10)

# 스위치 변경
switch_frame("searchIframe")
time.sleep(1)
# 버튼 변수들
next_btn = browser.find_elements(By.CSS_SELECTOR, '.zRM9F> a')
browser.implicitly_wait(10)

# 버튼길이만큼 반복 첫번재는 이전버튼 이므로 제외
for btn in range(len(next_btn))[1:]: 
    print(f'------------------------------------{btn}/{len(next_btn)-1}------------------------------------')

    #body부분을 잡기 위해 쓸데없는 버튼을 클릭해줌
    browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/div[1]/div').click()

    #검색결과가 모두 보이지 않기 때문에 page down을 눌러 끝까지 펼쳐준다.
    for scroll_1 in range(8):
        browser.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

    resp = browser.page_source     # 처음 브라우저 소스와, 버튼을 누를때마다 그것에 관한 페이지 소스 가져옴
    soup = BeautifulSoup(resp, 'html.parser') # 브라우저소스를 html로 변환하여 가져옴
    length = len(soup.select('#_pcmap_list_scroll_container > ul > li'))+1  # 한페이지에 보이는 카페들의 개수 반환

    # 카페들의 개수만큼 반복
    for j in range(1, length+1):
        # 스위치 변경
        switch_frame("searchIframe")

        # 해당하는 카페에 정보를 알기위해 그 카페를 클릭해줌
        # 구마다 /html/body/div[3]/div/div[3]....../sapn[1] or /html/body/div[3]/div/div[4]....../sapn[1] 가 있음
        try:
            browser.find_element(By.XPATH, f'/html/body/div[3]/div/div[3]/div[1]/ul/li[{j}]/div[1]/a/div/div/span[1]').click()
        except NoSuchElementException:
            browser.find_element(By.XPATH, f'/html/body/div[3]/div/div[4]/div[1]/ul/li[{j}]/div[1]/a/div/div/span[1]').click()
        finally:
            # 카페정보를 받아오는 시간 충분하게 받아줌 인터넷이 느려 충분히 시간 줘야됌
            time.sleep(3)

        # 스위치 변경 
        switch_frame("entryIframe")

        # 버튼이 누를 필요 없어서 제거
#---------------------------------------------------------------------------------------
#             # 가끔 버튼을 찾지 못함
#             try:
#                 time.sleep(1)
#                 browser.find_element(By.CLASS_NAME,'_UCia').click()
#                 browser.implicitly_wait(10)
#                 time.sleep(1)
#             # 같은 버튼에 이름만 다른 클래스임
#             except Exception as e:
#                 browser.find_element(By.CLASS_NAME,'PkgBl').click()
#                 browser.implicitly_wait(10)
#                 time.sleep(1)
#---------------------------------------------------------------------------------------

        print(f'-------------------------------{j}번째-------------------------------')

        # 카페 주소 저장
        cafe_addr = WebDriverWait(browser,1000).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/a/span[1]')))

        cafe_address = cafe_addr.text


        # 이미지 가져오기
        background_image_style = browser.find_element(By.XPATH,'/html/body/div[3]/div/div/div/div[1]/div/div[1]/div/a/div').get_attribute('style')
        background_image_url = re.search(r'url\("([^"]+)"\)', background_image_style).group(1)

        # 위에서 버튼을 누르지 않으니 스크롤이 제대로 돌아가지 않음 entryIframe 내의 element 클릭 그래야 스크롤이 내려감
        browser.find_element(By.ID, '_title').click()
        for scroll in range(0,10):
            browser.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

        resp = browser.page_source
        soup = BeautifulSoup(resp, 'html.parser')

        cafe_menu_ls = []
        cafe_menu = soup.select('div.MN48z > div.erVoL > div.MENyI > span.VQvNX')
        if len(cafe_menu) == 0:
            cafe_menu = soup.select('div.YzCTi > div.Fi0vA > div.RhpMT > a.place_bluelink.ihmWt')
            for menu in cafe_menu:
                cafe_menu_ls.append(menu.text)
        else:
            for menu in cafe_menu:
                cafe_menu_ls.append(menu.text)

        # 리뷰 높은순 3개 가져오기.
        cafe_review = []
        try:
            for k in range(1,4):
                cafe_review.append(soup.select(f'div.place_section_content > div.iqjjt > a > ul > li:nth-child({k}) > div.CsBE9 > span.nWiXa')[0].text[1:-1])
        except:
            cafe_review.append('리뷰 참여자가 10명이 되지 않습니다.')

        # 스위치 변경
        switch_frame("searchIframe")

        resp = browser.page_source
        soup = BeautifulSoup(resp, 'lxml')

# --------------------------------------------------------평점이 없는 매장이 훨신 많아 평점 제거--------------------------------------------------
        # 딕셔너리에 담을 변수들 생성
        cafe_name = soup.select(f'#_pcmap_list_scroll_container > ul > li:nth-child({j}) > div.CHC5F > a.tzwk0 > div > div > span.TYaxT')[0].text
        cafe_classification = soup.select(f'#_pcmap_list_scroll_container > ul > li:nth-child({j}) > div.CHC5F > a.tzwk0 > div > div > span.KCMnt')[0].text
        cafe_review_count = soup.select(f'#_pcmap_list_scroll_container > ul > li:nth-child({j}) > div.CHC5F > div > div > span:nth-child(2)')[0].text
        if cafe_review_count[:2] == '리뷰':
            cafe_review_count = cafe_review_count[3:]
        elif cafe_review_count[:2] != '리뷰':
            try:
                cafe_review_count = soup.select(f'#_pcmap_list_scroll_container > ul > li:nth-child({j}) > div.CHC5F > div > div > span:nth-child(3)')[0].text[3:]
            except:

                cafe_review_count = soup.select(f'#_pcmap_list_scroll_container > ul > li:nth-child({j}) > div.CHC5F > div > div > span:nth-child(1)')[0].text[3:]
        else :
            cafe_review_count = '0'

        cafe_name_list.append(cafe_name)
        cafe_classification_list.append(cafe_classification)
        cafe_review_count_list.append(cafe_review_count)
        cafe_address_list.append(cafe_address)
        cafe_review_list.append(cafe_review)
        background_image_url_list.append(background_image_url)
        cafe_menu_ls_list.append(cafe_menu_ls)

        dict_temp = {
            'name':cafe_name_list,
            'class':cafe_classification_list,
            'review_count':cafe_review_count_list,
            'address':cafe_address_list,
            'review_top3':cafe_review_list,
            'background_image_url':background_image_url_list,
            'cafe_menu':cafe_menu_ls_list        
        }    
        
    next_btn[-1].click() # 다음 버튼 클릭
    time.sleep(5)
    # 버튼이 눌리지 않으면 종료
    if not next_btn[-1].is_enabled():
        break

browser.implicitly_wait(10)
browser.close()

df = pd.DataFrame(dict_temp)
df.to_csv('C:/Users/user/crawling_project/crawling_강남구.csv',index = False, encoding='utf-8-sig')


------------------------------------1/6------------------------------------
-------------------------------1번째-------------------------------
-------------------------------2번째-------------------------------
-------------------------------3번째-------------------------------
-------------------------------4번째-------------------------------
-------------------------------5번째-------------------------------
-------------------------------6번째-------------------------------
-------------------------------7번째-------------------------------
-------------------------------8번째-------------------------------
-------------------------------9번째-------------------------------
-------------------------------10번째-------------------------------
-------------------------------11번째-------------------------------
-------------------------------12번째-------------------------------
-------------------------------13번째-------------------------------
-------------------------------14번째---------------------------

NameError: name 'NoSuchElementException' is not defined

In [25]:
import pandas as pd
import glob

# 파일 경로
file_path = 'C:/Users/user/crawling_project'

# 파일 경로 패턴을 사용하여 여러 파일을 선택
files = glob.glob(file_path + '/*구.csv')

# 각 파일을 DataFrame으로 읽어와 리스트에 추가
dfs = [pd.read_csv(file) for file in files]

# 모든 DataFrame을 합치기
result = pd.concat(dfs, ignore_index=True)

# 결과를 새로운 파일로 저장
result.to_csv(file_path + '/crawling_서울시_자치구_카페_조사.csv', index=False)

In [31]:
result.isna().sum()

name                    0
class                   0
review_count            0
address                 1
review_top3             0
background_image_url    0
cafe_menu               0
dtype: int64

In [33]:
nan_rows = result[result['address'].isna()]

# 출력
print(nan_rows)

             name class review_count address                      review_top3  \
2076  파리바게뜨 구로구청점  베이커리         999+     NaN  ['빵이 맛있어요', '친절해요', '매장이 청결해요']   

                                   background_image_url  \
2076  https://search.pstatic.net/common/?autoRotate=...   

                                              cafe_menu  
2076  ['러브초', '생딸기프레지에 (15cm)', '꾸안꾸 치즈 아이스', '[NEW]...  


In [26]:
gu = []
for i in range(len(df.address)):
    if pd.notnull(df.address[i]):  # NaN이 아닌 경우에만 슬라이싱
        gu.append(df.address[i][3:6])
    else:
        gu.append('')  # NaN인 경우 빈 문자열 또는 다른 값을 설정

g = pd.DataFrame({'gu': gu})
df = pd.concat([df, g], axis=1)

df = df[['name', 'gu', 'address', 'class', 'cafe_menu', 'review_count', 'review_top3', 'background_image_url']]

# save_path/filename.csv 입력
df.to_csv('C:/Users/user/crawling_project/crawling_서울시_자치구_카페_조사_최종.csv', index=False, encoding='utf-8-sig')


In [28]:
df["address"== NaN ]

NameError: name 'NaN' is not defined